# L09 Soluzioni

In [ ]:
from itertools import product, chain

from liblet import (
    Grammar, 
    Production, 
    Derivation, 
    ProductionGraph, 
    prods2table,
    side_by_side
)

from L06 import (
    eliminate_unit_rules, 
    eliminate_ε_rules, 
    transform_nonsolitary,
    make_binary,
    cyk
)

# Tutte le derivazioni leftmost (nella grammatica originale)

Iniziamo da `all_derives(ω, i, l)` che (in analogia a `derives(ω, i, l)`) determini se la forma sentenziale ω deriva una sottoparola dell'input e qualora lo faccia, restituisca una lista di tutte le possibili suddivisioni in cui la sottoparola può essere derivata dai simboli di ω.

In [ ]:
def make_all_derives(R, INPUT): 
    def all_derives(ω, i, l):
        if not ω or ('ε', ) == ω: return (l == 0, [[]])
        res = []
        X, *χ = ω
        if X in G.T:
            if i <= len(INPUT) and X == INPUT[i - 1]:
                d, ss = all_derives(χ, i + 1, l - 1)
                if d: res.extend([1] + s for s in ss)
        else:
            for k in range(0, l + 1):
                if X in R[(i, k)]:
                    d, ss = all_derives(χ, i + k, l - k)
                    if d: res.extend([k] + s for s in ss)
        return (True, res) if res else (False, [[]])
    return all_derives

In [ ]:
# per fare un test consideriamo una grammatica (ambigua) più semplice

G = Grammar.from_string("""
E -> E + E | E * E | e
""")

Gp_cnf = transform_nonsolitary(make_binary(eliminate_unit_rules(eliminate_ε_rules(G))))
prods2table(Gp_cnf)

Una domanda, ma `Gp_cnf` è ancora ambigua?

In [ ]:
# questo input ammette due alberi distinti: 
# uno con il + sopra il * e l'altro viceversa

INPUT = 'e+e*e'

Rp = cyk(Gp_cnf, INPUT)

Rε = {A for A in Gp_cnf.N if ('ε', ) in Gp_cnf.alternatives(A)}
for i in range(1, len(INPUT) + 2): Rp[(i, 0)] = Rε
    
Rp

In [ ]:
# vediamo se all_derives se ne accorge 

all_derives = make_all_derives(Rp, INPUT) 

all_derives(['E1', 'E'], 1, len(INPUT))

(True, [[2, 3], [4, 1]])

Questa risposta dice che `E1 E` può derivare l'input in due modi: in particolare che `E1` può limitarsi a produrre i due simboli `e+` e quindi `E` può produrre la moltiplicazione, ossia i restanti `e*e`, oppure che `E1` produrrà la somma, ossia i primi quattro simboli `e +e*` e quindi `E` produrrà il terminale `e`.

La costruzione di `get_all_original_leftmost_prods` dovrà determinare (con `all_derive`) le possibili suddivisioni della sottoparola e combinare le derivazioni che otterrà a partire da esse tramite un prodotto cartesiano (come illustrato negli handout).

In [ ]:
def get_all_original_leftmost_prods(G, all_derives, N):
    def all_prods(X, i, l):
        if X in G.T: return [[]]
        res = []
        for A, α in filter(Production.such_that(lhs = X), G.P):
            d, sps = all_derives(α, i, l)
            if not d: continue
            for sp in sps:
                Bprods = []
                bi = i
                for B, li in zip(α, sp): 
                    Bprods.append(all_prods(B, bi, li))
                    bi += li
                for spx in map(list, map(chain.from_iterable, product(*Bprods))):
                    res.append([G.P.index(Production(A, α))] + spx)
        return res
    return all_prods(G.S, 1, N)

In [ ]:
# otteniamo tutte le possibili derivazioni (leftmost)

all_prods = get_all_original_leftmost_prods(G, all_derives, len(INPUT))

In [ ]:
# questi sono i due alberi di parsing per l'input

side_by_side(*[ProductionGraph(Derivation(G).leftmost(prods)) for prods in all_prods])